In [18]:
import h5py
import matplotlib.pyplot as plt
import mujoco
import dm_control
import numpy as np
from dm_control import mjcf
from copy import deepcopy
from dm_control.mujoco.wrapper.mjbindings import enums
import imageio

### Read the provided h5 file from Eric and shorten it to have 1000 data points from index 7000 - 8000 of the original data file

In [19]:

with h5py.File("./tests/data/points3d_transformed.h5", 'r') as f:
    # print(f.keys())
    dset = f["tracks"]
    print(f'shape of original file {dset.shape}')

    shorten_file = h5py.File('./tests/data/points3d_transformed_shorten.h5', 'w')
    shorten_file.create_dataset('tracks', data=dset[7000:8000,])
    shorten_file.close()

shape of original file (123548, 1, 3, 3)


In [20]:
# sanity check for the new data file

with h5py.File("./tests/data/points3d_transformed_shorten.h5", 'r') as f:
    print(f'shape of shortened file {f["tracks"].shape}')

shape of shortened file (1000, 1, 3, 3)


### Run Stac MJX

In [21]:
import stac_mjx 
from pathlib import Path

# Enable XLA flags if on GPU
stac_mjx.enable_xla_flags()

# Choose parent directory as base path for data files
base_path = Path("/root/vast/joshua/stac-mjx")

# Load configs
cfg = stac_mjx.load_configs(base_path / "configs")
# cfg = stac_mjx.load_configs(base_path + "/configs")

# Load data
kp_data, sorted_kp_names = stac_mjx.load_mocap(cfg, base_path)

# Run stac
fit_path, ik_only_path = stac_mjx.run_stac(
 cfg,
 kp_data, 
 sorted_kp_names, 
 base_path=base_path
)

Config loaded and validated.


/root/vast/joshua/stac-mjx/stac_mjx/utils.py:16: DeprecationWarning: jax.lib.xla_bridge.get_backend is deprecated; use jax.extend.backend.get_backend.
  if xla_bridge.get_backend().platform == "gpu":


Running fit. Mocap data shape: (1, 9)


KeyboardInterrupt: 

In [10]:
fit_path, ik_only_path
meshes/stl/Bone_Scapula_right_v3.stl

(PosixPath('/root/vast/joshua/stac-mjx/mouse_arm_fit.p'), None)

In [12]:
import os
# os.path.isfile('/root/vast/joshua-akira-stac/stac-mjx/meshes/stl/Bone_Scapula_right_v3.stl')
os.path.isdir('/root/vast/joshua-akira-stac/')


False

In [6]:
with h5py.File("mouse_arm_fit.h5", 'r') as f:
    print(f.keys())
    for i in f.keys():
        print(f[i])
    # print(f'shape of shortened file {f["kp_data"].shape}')
    # print(f["kp_data"])
    # print()
    # print('kp_names')
    # print(list(f['kp_names']))

    # print('\nqpos')
    # print(list(f['qpos']))

    # print('\nxpos')
    # print(list(f['xpos']))

<KeysViewHDF5 ['config', 'kp_data', 'kp_names', 'marker_sites', 'names_qpos', 'names_xpos', 'offsets', 'qpos', 'qvel', 'xpos', 'xquat']>
<HDF5 dataset "config": shape (), type "|S952">
<HDF5 dataset "kp_data": shape (1, 9), type "<f4">
<HDF5 dataset "kp_names": shape (3,), type "|S8">
<HDF5 dataset "marker_sites": shape (1, 3, 3), type "<f4">
<HDF5 dataset "names_qpos": shape (4,), type "|S12">
<HDF5 dataset "names_xpos": shape (8,), type "|S8">
<HDF5 dataset "offsets": shape (3, 3), type "<f4">
<HDF5 dataset "qpos": shape (1, 4), type "<f4">
<HDF5 dataset "qvel": shape (0,), type "<f8">
<HDF5 dataset "xpos": shape (1, 8, 3), type "<f4">
<HDF5 dataset "xquat": shape (1, 8, 4), type "<f4">


In [7]:
with h5py.File("mouse_arm_ik_only.h5", 'r') as f:
    print(f.keys())
    for i in f.keys():
        print(f[i])
    # print(f'shape of shortened file {f["kp_data"].shape}')
    # print(f["kp_data"])
    # print()
    # print('kp_names')
    # print(list(f['kp_names']))

    # print('\nqpos')
    # print(list(f['qpos']))

    # print('\nxpos')
    # print(list(f['xpos']))

<KeysViewHDF5 ['config', 'kp_data', 'kp_names', 'marker_sites', 'names_qpos', 'names_xpos', 'offsets', 'qpos', 'qvel', 'xpos', 'xquat']>
<HDF5 dataset "config": shape (), type "|S952">
<HDF5 dataset "kp_data": shape (1000, 9), type "<f4">
<HDF5 dataset "kp_names": shape (3,), type "|S8">
<HDF5 dataset "marker_sites": shape (1000, 3, 3), type "<f4">
<HDF5 dataset "names_qpos": shape (4,), type "|S12">
<HDF5 dataset "names_xpos": shape (8,), type "|S8">
<HDF5 dataset "offsets": shape (3, 3), type "<f4">
<HDF5 dataset "qpos": shape (1000, 4), type "<f4">
<HDF5 dataset "qvel": shape (0,), type "<f8">
<HDF5 dataset "xpos": shape (1000, 8, 3), type "<f4">
<HDF5 dataset "xquat": shape (1000, 8, 4), type "<f4">


In [4]:
config, render = stac_mjx.viz_stac(
    data_path="mouse_arm_ik_only.h5",
    n_frames=100,
    save_path='mouse_arm_ik_only_video.mp4'
)

0it [00:00, ?it/s]/root/miniforge3/envs/stac-mjx-env/lib/python3.11/subprocess.py:1885: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
1000it [02:58,  5.60it/s]


### Okay I have no idea what is going on with the video output so now I am trying to go through Charles' code line by line and try to replicate his coding process 

##### First is to get the stac object and view it in the 3D space

In [2]:
from pathlib import Path
import stac_mjx

In [4]:

cfg = stac_mjx.load_configs(Path.cwd() / "configs")

# Load data
kp_data, kp_names = stac_mjx.load_mocap(cfg, Path.cwd())
stac = stac_mjx.stac.Stac(Path.cwd() / cfg.model.MJCF_PATH, cfg, kp_names)

_, d = stac_mjx.io.load_stac_data("./mouse_arm_ik_only_torquearm.h5")
qposes = d.qpos

Config loaded and validated.


In [6]:
qposes.shape

(1000, 4)

In [7]:
mj_model, body_site_idx, kp_site_idx = stac._create_keypoint_sites()

In [ ]:
# take a look through some variables.. i dont know what the last line is
# but i assume its a list of the different objects in the mujocuo model
# i'm just confused why shoulder_kp, elbow_kp, and wrist_kp are duplicated
print(cfg.model.keys())
print(f'body index sites: {body_site_idx}')
mjcf.Physics.from_mjcf_model(stac._root).named.model.site_pos

dict_keys(['MJCF_PATH', 'FTOL', 'ROOT_FTOL', 'LIMB_FTOL', 'N_ITERS', 'KP_NAMES', 'KEYPOINT_MODEL_PAIRS', 'KEYPOINT_INITIAL_OFFSETS', 'ROOT_OPTIMIZATION_KEYPOINT', 'TRUNK_OPTIMIZATION_KEYPOINTS', 'KEYPOINT_COLOR_PAIRS', 'SCALE_FACTOR', 'MOCAP_SCALE_FACTOR', 'SITES_TO_REGULARIZE', 'RENDER_FPS', 'N_SAMPLE_FRAMES', 'M_REG_COEF'])
body index sites: [10, 27, 35]


FieldIndexer(site_pos):
                               x         y         z         
 0               Shoulder_kp [-0.000357  0.000128 -0.000263]
 1                  Elbow_kp [ 3.83e-05 -0.000423  0.000135]
 2                  Wrist_kp [-3.74e-05 -0.000361 -4.13e-06]
 3              Pec_C_Origin [-0.002     0         0.0005  ]
 4                 AD_Origin [-0         0.0013    0.0007  ]
 5                Lat_Origin [-0.0005   -0.004    -0.0035  ]
 6                 PD_Origin [ 0.0004   -0.0007    0.0006  ]
 7                 MD_Origin [ 0.0007    0.0005    0.0005  ]
 8       Triceps_Long_Origin [ 0        -0.0003   -0.0005  ]
 9        Biceps_Long_Origin [ 0.0001    0.0003    0.0007  ]
10                  Shoulder [ 0         0         0       ]
11  shoulder_joint_viz_large [ 0         0         0       ]
12   interial_center_humerus [-0.0005    0.0003   -0.004   ]
13                  Pec_C_P2 [-0.0005    0.0009   -0.0003  ]
14           Pec_C_Insertion [-0         0.001    -0.0005  ]

In [9]:
scene_option = mujoco.MjvOption()
scene_option.geomgroup[1] = 0
scene_option.geomgroup[2] = 1
scene_option.sitegroup[2] = 1
scene_option.sitegroup[3] = 0
scene_option.flags[enums.mjtVisFlag.mjVIS_TRANSPARENT] = True
scene_option.flags[enums.mjtVisFlag.mjVIS_LIGHT] = True
scene_option.flags[enums.mjtVisFlag.mjVIS_CONVEXHULL] = True
scene_option.flags[enums.mjtRndFlag.mjRND_SHADOW] = True
scene_option.flags[enums.mjtRndFlag.mjRND_REFLECTION] = True
scene_option.flags[enums.mjtRndFlag.mjRND_SKYBOX] = True
scene_option.flags[enums.mjtRndFlag.mjRND_FOG] = True

In [32]:
physics = mjcf.Physics.from_mjcf_model(stac._root)
render_mj_model = deepcopy(physics.model.ptr)

mj_data = mujoco.MjData(render_mj_model)
mujoco.mj_kinematics(render_mj_model, mj_data)
renderer = mujoco.Renderer(render_mj_model, height=1200, width=1920)
# renderer = mujoco.Renderer(render_mj_model)
# renderer._height=1200

In [33]:
render_mj_model.site_pos[kp_site_idx].shape

(3, 3)

In [ ]:
with imageio.get_writer("test.png", fps=20) as video:

    # original code from charles' is:
    # render_mj_model.site_pos[keypoint_site_idxs] = np.reshape(kps, (-1, 3))
    # my code: (just using random numbers)
    render_mj_model.site_pos[kp_site_idx] = np.array([0, 0, 1, 0, 0, 50, 0, 0, 0]).reshape((3, 3))
    # mj_data.qpos = [0, 1, 0, 0] # random numbers

    mujoco.mj_fwdPosition(render_mj_model, mj_data)

    renderer.update_scene(mj_data, camera=0, scene_option=scene_option)
    video.append_data(renderer.render())

/root/miniforge3/envs/stac-mjx-env/lib/python3.11/site-packages/imageio/plugins/pillow.py:409: DeprecationWarning: The keyword `fps` is no longer supported. Use `duration`(in ms) instead, e.g. `fps=50` == `duration=20` (1000 * 1/50).
  warnings.warn(


##### todo1: get rid of the kp sites from the physics thing and run it through the input h5 data
##### todo2: try to find a way to directly change with the joint angles
##### todo3: do the Umeyama registration 

## todo 1

In [10]:
stac2 = stac_mjx.stac.Stac(Path.cwd() / cfg.model.MJCF_PATH, cfg, kp_names)
physics2 = mjcf.Physics.from_mjcf_model(stac2._root)
render_mj_model2 = deepcopy(physics2.model.ptr)

In [ ]:
mjcf.Physics.from_mjcf_model(stac2._root).named.model.site_pos

FieldIndexer(site_pos):
                               x         y         z         
 0              Pec_C_Origin [-0.002     0         0.0005  ]
 1                 AD_Origin [-0         0.0013    0.0007  ]
 2                Lat_Origin [-0.0005   -0.004    -0.0035  ]
 3                 PD_Origin [ 0.0004   -0.0007    0.0006  ]
 4                 MD_Origin [ 0.0007    0.0005    0.0005  ]
 5       Triceps_Long_Origin [ 0        -0.0003   -0.0005  ]
 6        Biceps_Long_Origin [ 0.0001    0.0003    0.0007  ]
 7                  Shoulder [ 0         0         0       ]
 8  shoulder_joint_viz_large [ 0         0         0       ]
 9   interial_center_humerus [-0.0005    0.0003   -0.004   ]
10                  Pec_C_P2 [-0.0005    0.0009   -0.0003  ]
11           Pec_C_Insertion [-0         0.001    -0.0005  ]
12                    Lat_P2 [-0.001     0        -0.0006  ]
13             Lat_Insertion [-0.0004    0.0008   -0.0006  ]
14                     AD_P2 [ 0.0003    0.0012   -0.0003  ]

In [12]:
mj_data2 = mujoco.MjData(render_mj_model2)
mujoco.mj_kinematics(render_mj_model2, mj_data2)
renderer2 = mujoco.Renderer(render_mj_model2, height=1200, width=1920)

In [22]:
with imageio.get_writer("input_data_arm_movement_(attempt2).mp4", fps=20) as video:
    for i in range(1000):
        render_mj_model2.site_pos[[7, 24, 32]] = kp_data[i].reshape(-1, 3)
        # mj_data2.qpos = [i, i, i, i]
        mj_data2.qpos = qposes[i]

        mujoco.mj_fwdPosition(render_mj_model2, mj_data2)

        renderer2.update_scene(mj_data2, camera=0, scene_option=scene_option)
        video.append_data(renderer2.render())

In [26]:
render_mj_model2.site_pos[[7, 24, 32]]

array([[-0.00723403,  0.00966849,  0.0131236 ],
       [-0.01064992,  0.01103032,  0.00484736],
       [-0.00338147,  0.00977638,  0.00280233]])

In [ ]:
mjcf.Physics.from_mjcf_model(stac2._root).named.model.site_pos

FieldIndexer(site_pos):
                               x         y         z         
 0              Pec_C_Origin [-0.002     0         0.0005  ]
 1                 AD_Origin [-0         0.0013    0.0007  ]
 2                Lat_Origin [-0.0005   -0.004    -0.0035  ]
 3                 PD_Origin [ 0.0004   -0.0007    0.0006  ]
 4                 MD_Origin [ 0.0007    0.0005    0.0005  ]
 5       Triceps_Long_Origin [ 0        -0.0003   -0.0005  ]
 6        Biceps_Long_Origin [ 0.0001    0.0003    0.0007  ]
 7                  Shoulder [ 0         0         0       ]
 8  shoulder_joint_viz_large [ 0         0         0       ]
 9   interial_center_humerus [-0.0005    0.0003   -0.004   ]
10                  Pec_C_P2 [-0.0005    0.0009   -0.0003  ]
11           Pec_C_Insertion [-0         0.001    -0.0005  ]
12                    Lat_P2 [-0.001     0        -0.0006  ]
13             Lat_Insertion [-0.0004    0.0008   -0.0006  ]
14                     AD_P2 [ 0.0003    0.0012   -0.0003  ]

In [37]:
dir(mj_data)

['B_colind',
 'B_rowadr',
 'B_rownnz',
 'D_colind',
 'D_rowadr',
 'D_rownnz',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_address',
 'act',
 'act_dot',
 'actuator',
 'actuator_force',
 'actuator_length',
 'actuator_moment',
 'actuator_velocity',
 'body',
 'bvh_aabb_dyn',
 'bvh_active',
 'cacc',
 'cam',
 'cam_xmat',
 'cam_xpos',
 'camera',
 'cdof',
 'cdof_dot',
 'cfrc_ext',
 'cfrc_int',
 'cinert',
 'contact',
 'crb',
 'ctrl',
 'cvel',
 'dof_island',
 'dof_islandind',
 'efc_AR',
 'efc_AR_colind',
 'efc_AR_rowadr',
 'efc_AR_rownnz',
 'efc_D',
 'efc_J',
 'efc_JT',
 'efc_JT_colind',
 'efc_JT_rowadr',
 'efc_JT_rownnz',
 'efc_JT_rowsup

In [45]:
mj_data2.qpos

array([0., 0., 0., 0.])